# SROP Winter 2025

### Dataset loading

In [3]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import tqdm
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
from torchsummary import summary
import struct

In [4]:
DEVICE = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

Using device: cpu


In [5]:
train_dataset = torchvision.datasets.FashionMNIST(
    'data/', train=True, download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
    ])
)
test_dataset = torchvision.datasets.FashionMNIST(
    'data/', train=False, download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
    ])
)

# Create a 10% validation set using stratification.
train_indices, val_indices, _, _ = train_test_split(
    range(len(train_dataset)),
    train_dataset.targets,
    stratify=train_dataset.targets,
    test_size=0.1
)
train_split = Subset(train_dataset, train_indices)
val_split = Subset(train_dataset, val_indices)

# Update the training set transform to include data augmentation.
augment_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomAffine(degrees=5, translate=(0.05, 0.05), scale=(0.95, 1.05), shear=0.05),
    torchvision.transforms.ColorJitter(brightness=0.1, contrast=0.1),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081,))
])
train_split.dataset.transform = augment_transform

# Create DataLoader objects.
train_batch_size = 512
test_batch_size  = 256
train_batches = DataLoader(train_split, batch_size=train_batch_size, shuffle=True)
val_batches   = DataLoader(val_split, batch_size=train_batch_size, shuffle=True)
test_batches  = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

print("Number of training batches:", len(train_batches))
print("Number of validation batches:", len(val_batches))
print("Number of test batches:", len(test_batches))

Number of training batches: 106
Number of validation batches: 12
Number of test batches: 40


Data is now formatted and loaded.


AI-generated code to set printing color

In [6]:
def set_color(r, g, b, bg=False):
    """
    Sets the text or background color using RGB values.

    Args:
        r: Red value (0-255).
        g: Green value (0-255).
        b: Blue value (0-255).
        bg: If True, sets the background color; otherwise, sets the text color.
    """
    type = 48 if bg else 38
    print(f"\033[{type};2;{r};{g};{b}m", end="")

In [ ]:
with open('data/FashionMNIST/raw/train-images-idx3-ubyte','rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    # print(data.shape) # (47040000,)
    Xtraindata = np.transpose(data.reshape((size, nrows*ncols)))

with open('data/FashionMNIST/raw/train-labels-idx1-ubyte','rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    Ylabeldata = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    # print(data.shape) # (47040000,)

imgIndex = 0

for row in range(28):
    for cell in range(28):
        pixelBrightness = Xtraindata[row*28+cell][imgIndex]
        set_color(pixelBrightness,pixelBrightness,pixelBrightness,True)
        print(' * ',end='')
    print()
print(Ylabeldata[imgIndex])

In [30]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
reduced_data = pca.fit_transform(np.transpose(Xtraindata))
print(reduced_data.shape)


(60000, 2)


AI-generated code to visualize the PCA reduced data

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=Ylabeldata, cmap='viridis', s=5)
plt.colorbar(label='Label')
plt.title('PCA Reduced Data')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

In [ ]:
with open('data/FashionMNIST/raw/t10k-images-idx3-ubyte','rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    # print(data.shape) # (47040000,)
    Xtestdata = np.transpose(data.reshape((size, nrows*ncols)))

with open('data/FashionMNIST/raw/t10k-labels-idx1-ubyte','rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    Ytestlabel = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    # print(data.shape) # (47040000,)